In [1]:
!pip install -q gradio scikit-learn pandas


In [2]:
import pandas as pd

# Simulated network traffic data (for demonstration)
data = {
    "duration": [1, 10, 0.2, 50, 5, 0.1, 0.5, 25, 100, 2],
    "protocol": [0, 1, 0, 1, 0, 0, 1, 1, 1, 0],  # 0 = TCP, 1 = UDP
    "packet_length": [100, 300, 50, 500, 200, 30, 70, 400, 600, 150],
    "src_bytes": [200, 1000, 20, 1500, 300, 15, 50, 1100, 2000, 180],
    "dst_bytes": [150, 800, 10, 1400, 250, 10, 30, 900, 1900, 160],
    "flag_syn": [1, 0, 1, 0, 0, 1, 1, 0, 0, 1],
    "label": [
        "normal", "malicious", "suspicious", "malicious", "normal",
        "suspicious", "suspicious", "malicious", "malicious", "normal"
    ]
}

df = pd.DataFrame(data)


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Encode labels
le = LabelEncoder()
df["label_encoded"] = le.fit_transform(df["label"])

# Features and labels
X = df.drop(columns=["label", "label_encoded"])
y = df["label_encoded"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("📊 Classification Report:\n", classification_report(
    y_test,
    y_pred,
    labels=[0, 1, 2],
    target_names=le.classes_,
    zero_division=0  # avoids division by zero warning
))



📊 Classification Report:
               precision    recall  f1-score   support

   malicious       1.00      1.00      1.00         2
      normal       0.00      0.00      0.00         0
  suspicious       1.00      1.00      1.00         1

    accuracy                           1.00         3
   macro avg       0.67      0.67      0.67         3
weighted avg       1.00      1.00      1.00         3



In [5]:
import gradio as gr

def predict_traffic(duration, protocol, packet_length, src_bytes, dst_bytes, flag_syn):
    input_data = [[duration, protocol, packet_length, src_bytes, dst_bytes, flag_syn]]
    pred = model.predict(input_data)[0]
    label = le.inverse_transform([pred])[0]
    return f"🧠 Prediction: {label.upper()}"

iface = gr.Interface(
    fn=predict_traffic,
    inputs=[
        gr.Number(label="Duration (s)"),
        gr.Radio([0, 1], label="Protocol (0=TCP, 1=UDP)"),
        gr.Number(label="Packet Length"),
        gr.Number(label="Source Bytes"),
        gr.Number(label="Destination Bytes"),
        gr.Checkbox(label="SYN Flag")
    ],
    outputs=gr.Textbox(label="Prediction"),
    title="AI Network Traffic Classifier",
    description="Enter network packet features and detect if it's Normal, Suspicious, or Malicious."
)

iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8e260ca881770ee90a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
